In [1]:
import pandas as pd
import numpy as np

import dask
import dask.dataframe as dd
import dask_ml

from sklearn.linear_model import LinearRegression, Ridge, Lasso, Huber
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import KFold, train_test_split

In [2]:
def train_val_score(trainX, trainy, valX, valy, model): 
    '''
    Given a train and validation set, generate the square root mean squared error for the train and validation set
    '''
    #first we fit the train set and get predictions and compare them to the actual
    model.fit(trainX, trainy)
    trainpredict = model.predict(trainX)
    trainmse = mean_squared_error(trainy, trainpredict)
    train_e = np.sqrt(trainmse)   
    
    #next we fit the validation set and get the predcitions and compare them to the actual
    model.fit(valX,valy)
    valpredict = model.predict(valX)
    valmse = mean_squared_error(valy, valpredict)
    val_e = np.sqrt(valmse)
    
    
    return print(str(round(train_e,6)) + ' Training Error, ' + str(round(val_e,6)) + ' Valid error')

In [3]:
data = pd.read_pickle('/home/michael/Documents/Projects/divvydataproject/data/fulldataagg.pkl')
data = data[data['datetime'].dt.year > 2013]
data = dd.from_pandas(data, npartitions = 8)

In [4]:
data.head()

,datetime,to_station_id,temp,precip,trips,location_soldierfield,location_unitedcenter,location_uscellular,location_wrigley
318445,2014-01-01,135,10.0,0.04,1.0,0.0,0.0,0.0,0.0
318446,2014-01-01,135,10.0,0.00,1.0,0.0,0.0,0.0,0.0
318447,2014-01-01,135,10.0,0.00,1.0,0.0,0.0,0.0,0.0
318448,2014-01-01,339,10.0,0.04,1.0,0.0,0.0,0.0,0.0
318449,2014-01-01,339,10.0,0.00,1.0,0.0,0.0,0.0,0.0


In [5]:
df_trainval, df_test = data.random_split([0.9, 0.1], random_state=2)
df_train, df_val = df_trainval.random_split([0.9, 0.1], random_state=2)

In [6]:
trainX = df_train[['temp','precip','location_soldierfield','location_unitedcenter','location_uscellular','location_wrigley']]
valX = df_val[['temp','precip','location_soldierfield','location_unitedcenter','location_uscellular','location_wrigley']]
trainy = df_train[['trips']]
valy = df_val[['trips']]

In [7]:
type(trainX)

dask.dataframe.core.DataFrame

In [ ]:
alpha = [1e-6, 1e-5,1e-4, 1e-3, 1e-2, 1e-1, 1]#, 1e1,1e2]
for alpha in alpha:
    lasso = Lasso(alpha=alpha, fit_intercept=True, random_state=77)
    print('Alpha: '+str(alpha))
    train_val_score(trainX, trainy, valX, valy, lasso).compute()

Alpha: 1e-06
